# 自动求导

标量的链式法则: <br><br>
&emsp;&emsp; $y = f(u), u = g(x)$<br><br>
&emsp;&emsp; $ \frac{\partial y}{\partial x} = \frac{\partial y}{\partial u} \frac{\partial u}{\partial x} $

拓展到向量:<br><br>
&emsp;&emsp; $\frac{\partial \vec{y}}{\partial \vec{x}} = \frac{\partial \vec{y}}{\partial \vec{u}} \frac{\partial \vec{u}}{\partial \vec{x}} $

## 自动求导

* 自动求导计算一个函数在指定值上的导数
* 它有别于
    * 符号求导(解析求导)
* 数值求导:
    * 通过数值拟合导数

## 计算图

* 将代码分解为操作子
* 将计算表示成一个无环图
* 显式构造
    * Tensorflow/Theano/MXNet
* 隐式构造:
    * PyTorch/MXNet

In [4]:
# 显式构造
from mxnet import sym

a = sym.var('a')
b = sym.var('b')
c = 2 * a + b

In [7]:
# 隐式构造
from mxnet import autograd, nd
with autograd.record():
    a = nd.ones((2, 1))
    b = nd.ones((2, 1))
    c = 2 * a + b

## 反向累积 

## 自动求导实现

求: $ y = 2x^Tx 的导数$

In [3]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

> 在计算y关于x的梯度之前, 我们需要一个地方来存储梯度 

In [9]:
x.requires_grad_(True)
x.grad

In [12]:
y = 2 * x.T @ x
y

tensor(28., grad_fn=<DotBackward0>)

> 通过调用反向传播函数来自动计算y关于x每个分量的梯度

&emsp;&emsp;&emsp;&emsp;
$\frac{\partial {y}}{\partial {\vec{x}}} =  4 \vec{x} $

In [13]:
y.backward()
x.grad 

tensor([ 0.,  4.,  8., 12.])

In [14]:
x.grad == 4 * x

tensor([True, True, True, True])

> 现在计算 x 的 另一个函数

In [15]:
# 默认情况下, PyTorch 会积累梯度， 我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

> 深度学习中, 我们的目的不是计算微分矩阵, 而是批量中每个样本单独计算的偏导数之和 

## 自由练习

In [4]:
import torch

$ y = 2x_1^3 + 3x_1x_2 + 4x_2$

In [6]:
x = torch.Tensor([1, 2])
x.requires_grad_(True)
y =  2*(x[0]**3) + 3*x[0]*x[1] + 4*x[1]

In [7]:
y.backward()
x.grad

tensor([12.,  7.])

In [52]:
# 让 x 和 y 分别下降, 学习率为 0.1
learning_rate = 0.1
x = torch.Tensor([11, 2.0])
x.requires_grad_(True)
ys = []
for i in range(1000):
    y = 2*(x[0]**2)  + 4*x[1]**2
    y.backward()
    if  len(ys) and y >= ys[-1]:
        break
    ys.append(y)
    print(y , i, x.grad )
    with torch.no_grad():
        x -= learning_rate * x.grad
    x.grad.zero_()

tensor(258., grad_fn=<AddBackward0>) 0 tensor([44., 16.])
tensor(87.7600, grad_fn=<AddBackward0>) 1 tensor([26.4000,  3.2000])
tensor(31.3888, grad_fn=<AddBackward0>) 2 tensor([15.8400,  0.6400])
tensor(11.2918, grad_fn=<AddBackward0>) 3 tensor([9.5040, 0.1280])
tensor(4.0647, grad_fn=<AddBackward0>) 4 tensor([5.7024, 0.0256])
tensor(1.4633, grad_fn=<AddBackward0>) 5 tensor([3.4214, 0.0051])
tensor(0.5268, grad_fn=<AddBackward0>) 6 tensor([2.0529e+00, 1.0240e-03])
tensor(0.1896, grad_fn=<AddBackward0>) 7 tensor([1.2317e+00, 2.0480e-04])
tensor(0.0683, grad_fn=<AddBackward0>) 8 tensor([7.3903e-01, 4.0960e-05])
tensor(0.0246, grad_fn=<AddBackward0>) 9 tensor([4.4342e-01, 8.1920e-06])
tensor(0.0088, grad_fn=<AddBackward0>) 10 tensor([2.6605e-01, 1.6384e-06])
tensor(0.0032, grad_fn=<AddBackward0>) 11 tensor([1.5963e-01, 3.2768e-07])
tensor(0.0011, grad_fn=<AddBackward0>) 12 tensor([9.5778e-02, 6.5536e-08])
tensor(0.0004, grad_fn=<AddBackward0>) 13 tensor([5.7467e-02, 1.3107e-08])
tensor(0.